# Imports

In [ ]:
import pandas as pd
from nn_spikes import NeuralNetwork, batchTrain, test
import spike_tools, utilities, KNN_spikes
from nn_spikes import getInputsAndTargets
import plotly.express as px

In [ ]:
data = pd.read_csv('./datasources/spikes/training_data.csv')
spikeLocations = pd.read_csv('./datasources/spikes/training_spike_locations.csv', index_col=0)

In [ ]:
data_training, data_validation, spikeIndexes_training, spikeIndexes_validation = spike_tools.dataPreProcess(data, spikeLocations, waveformWindow=100)

In [ ]:
#spike_tools.getAverageWaveforms(data_training, spikeIndexes_training[:200], classToPlot=3)

In [ ]:
#spike_tools.getAverageWaveforms(data_training, spikeIndexes_training[:200], classToPlot=2)

In [ ]:
#spike_tools.getAverageWaveforms(data_training, spikeIndexes_training[:200], classToPlot=1)

In [ ]:
#spike_tools.getAverageWaveforms(data_training, spikeIndexes_training[:200], classToPlot=0)

# Run Neural Network Classifier

In [ ]:
results = utilities.createResultsRepo([200, 500, 700, 900, 1100])

In [ ]:
for hid in results.keys():
    
    nn = NeuralNetwork(input_nodes=len(data_training.loc[spikeIndexes_training[0], 'waveform']), 
                       hidden_nodes=int(hid), 
                       output_nodes=4, 
                       lr=0.1,
                       error_function='difference-squared')

    nn, trainingCurve, validationCurve = batchTrain(data_training=data_training,
                                                              data_validation=data_validation,
                                                              spikeIndexes_training=spikeIndexes_training, 
                                                              spikeIndexes_validation=spikeIndexes_validation, 
                                                              nn=nn,
                                                              epochs=30,
                                                              plotCurves=False)
    results[hid]['nn'] = nn
    results[hid]['trainingCurve'] = trainingCurve
    results[hid]['validationCurve'] = validationCurve

In [ ]:
utilities.plotLearningCurves(results)

In [ ]:
waveforms = data_validation[data_validation['predictedSpike']==True]['waveform']
predictionsMLP = spike_tools.classifySpikesMLP(waveforms, nn)

In [ ]:
predictionsMLP

# Run SA